In [2]:
import pandas as pd
import numpy as np
from yahoo_fin import stock_info as finance

# Mock function to mimic the behavior of finance.get_data
def get_data(ticker, sdate, edate):
    # Assume this function returns a DataFrame similar to the one produced by finance.get_data
    # For demonstration, we're generating a DataFrame with a date range and mock data
    return finance.get_data(ticker, sdate, edate)


In [3]:
def identify_price_jumps(df, z_score_threshold=2, stability_threshold=1, lookback_period=90, lag_days_start=2, lag_days_end=5, 
                         include_negative_class=False, remove_duplicate_price_jump_dates=False):
    # Calculating daily price changes
    df['PriceChange'] = df['close'].pct_change()
    
    # Calculating rolling statistics
    rolling_mean = df['PriceChange'].rolling(window=lookback_period).mean()
    rolling_std = df['PriceChange'].rolling(window=lookback_period).std()
    
    # Identifying potential price jumps
    positive_results = []
    for i in range(lookback_period, len(df) - lag_days_end):
        current_z_score = (df.iloc[i]['PriceChange'] - rolling_mean[i]) / rolling_std[i]
        if abs(current_z_score) < stability_threshold:
            for lag_day in range(lag_days_start, lag_days_end+1):
                if i+lag_day < len(df):
                    future_price_change = df.iloc[i+lag_day]['PriceChange']
                    future_z_score = (future_price_change - rolling_mean[i+lag_day]) / rolling_std[i+lag_day]
                    if future_z_score > z_score_threshold:
                        positive_results.append({
                            'TriggerDate': df.index[i].date(),
                            'TriggerClose': df.iloc[i]['close'],
                            'PriceJumpDate': df.index[i+lag_day].date(),
                            'Close': df.iloc[i+lag_day]['close'],
                            'PriceJumpZScore': future_z_score,
                            'DaysAfterTrigger': lag_day,
                            'SignificantPriceJump': True,
                            'Ticker': df.iloc[0]['ticker']
                        })
                        break
                        
    # Convert positive results to DataFrame
    positive_results_df = pd.DataFrame(positive_results)

    # If remove_duplicate_price_jump_dates flag is set, remove duplicates
    if remove_duplicate_price_jump_dates:
        # Group by PriceJumpDate and randomly keep one record per group
        positive_results_df = positive_results_df.groupby('PriceJumpDate').apply(lambda x: x.sample(1)).reset_index(drop=True)


    # Sample negative class instances if required
    negative_results = []
    if include_negative_class:
        num_positive = len(positive_results_df)
        sampled_rows = df.iloc[lookback_period:len(df) - lag_days_end].sample(n=num_positive)
        for _, row in sampled_rows.iterrows():
            negative_results.append({
                'TriggerDate': row.name.date(),
                'TriggerClose': row['close'],
                'PriceJumpDate': None,  # No price jump date for negative class
                'Close': None,
                'PriceJumpZScore': None,
                'DaysAfterTrigger': None,
                'SignificantPriceJump': False,
                'Ticker': df.iloc[0]['ticker']
            })

    # Combine positive and negative results
    combined_results = pd.DataFrame(positive_results_df.to_dict('records') + negative_results).fillna(0)


    return combined_results

# Example usage
df = get_data('GME', '2019-01-01', '2023-11-15')
price_jumps_df = identify_price_jumps(df, z_score_threshold=2.5, stability_threshold=1, include_negative_class=True, remove_duplicate_price_jump_dates = True)
price_jumps_df.sample(5)


/tmp/ipykernel_9821/323460992.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_z_score = (df.iloc[i]['PriceChange'] - rolling_mean[i]) / rolling_std[i]
/tmp/ipykernel_9821/323460992.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  future_z_score = (future_price_change - rolling_mean[i+lag_day]) / rolling_std[i+lag_day]


,TriggerDate,TriggerClose,PriceJumpDate,Close,PriceJumpZScore,DaysAfterTrigger,SignificantPriceJump,Ticker
22,2020-06-15,1.1725,0,0.000000,0.000000,0.0,False,GME
1,2020-03-11,1.0350,2020-03-13,1.147500,3.288345,2.0,True,GME
13,2021-02-22,11.5000,2021-02-24,22.927500,3.827772,2.0,True,GME
11,2021-01-20,9.7800,2021-01-22,16.252501,4.066758,2.0,True,GME
41,2022-10-07,25.3500,0,0.000000,0.000000,0.0,False,GME


In [4]:
# Short sale volume

class ShortSaleVolume:
    
    bucket = 'mads-capstone-2023' 
    data_key = 'FINRA_Short_Sale_Data2.csv' 
    data_location = 's3://{}/{}'.format(bucket, data_key) 
    
    def __init__(self):
        self.has_downloaded = False 
    
    # Function to download data from S3
    def get_data(self):
        if not self.has_downloaded:
            self.df = pd.read_csv(self.data_location)
            self.df.drop(['Unnamed: 0'], axis = 1, inplace = True)
            self.df['shortvolume'] = self.df['totalvolume'] - self.df['shortvolume']
            self.df['longvolume'] = self.df['totalvolume'] - self.df['shortvolume']
            self.df['short_ratio'] = 1 - self.df['short_ratio']
            self.has_downloaded = True
        else:
            pass
        
    def short_ratio(self, ticker):
        # Check if data has been downloaded locally
        self.get_data()
        
        # Return data for ticker 
        df_ = self.df[self.df['ticker'] == ticker].copy()
        df_['date'] = pd.to_datetime(df_['date'])
        df_.set_index('date', inplace=True)
        return df_

    
    def calculate_short_ratio_means(self, ticker='GME', include_diff=False, include_roc=False):
        df = self.short_ratio(ticker)
        periods = [1, 5, 10, 21, 42]
        for period in periods:
            df[f'short_ratio_mean_{period}'] = df['short_ratio'].rolling(window=period).mean()

        if include_diff:
            for i in range(len(periods)):
                for j in range(i + 1, len(periods)):
                    df[f'short_ratio_mean_{periods[i]}_div_mean_{periods[j]}'] = df[f'short_ratio_mean_{periods[i]}'] / df[f'short_ratio_mean_{periods[j]}']
                    df[f'short_ratio_mean_{periods[j]}_div_mean_{periods[i]}'] = df[f'short_ratio_mean_{periods[j]}'] / df[f'short_ratio_mean_{periods[i]}']

        if include_roc:
            # print('working')
            for col in df.columns:
                if 'short_ratio_mean_' in col:
                    df[f'{col}_roc'] = df[col].pct_change() * 100

        result_df = df[[col for col in df.columns if 'short_ratio_mean_' in col or ('roc' in col and 'short_ratio_mean_' in col)]]
        return result_df

    def calculate_short_exempt_means(self, ticker='GME', include_diff=False, include_roc=False):
        df = self.short_ratio(ticker)
        periods = [1, 5, 10, 21, 42]

        df['short_exempt_percent'] = (df['shortexemptvolume'] / df['totalvolume'])

        for period in periods:
            df[f'short_exempt_percent_mean_{period}'] = df['short_exempt_percent'].rolling(window=period).mean()

        if include_diff:
            for i in range(len(periods)):
                for j in range(i + 1, len(periods)):
                    df[f'short_exempt_percent_mean_{periods[i]}_div_mean_{periods[j]}'] = df[f'short_exempt_percent_mean_{periods[i]}'] / df[f'short_exempt_percent_mean_{periods[j]}']
                    df[f'short_exempt_percent_mean_{periods[j]}_div_mean_{periods[i]}'] = df[f'short_exempt_percent_mean_{periods[j]}'] / df[f'short_exempt_percent_mean_{periods[i]}']

        if include_roc:
            for col in df.columns:
                if 'short_exempt_percent_mean_' in col:
                    df[f'{col}_roc'] = df[col].pct_change() * 100

        result_df = df[[col for col in df.columns if 'short_exempt_percent_mean_' in col or ('roc' in col and 'short_exempt_percent_mean_' in col)]]
        return result_df
    
    def calculate_investment_values(self, ticker='GME', days=20, investment_amount=1_000_000, include_diff=False, include_roc=False):
        df = self.short_ratio(ticker)

        # Retrieve closing prices from finance API
        closing_prices = finance.get_data(ticker, df.index.min(), df.index.max())['close']
        
        # Merge closing prices with short ratio data
        df = df.merge(closing_prices, left_index=True, right_index=True, how='left')

        # Ensure short_exempt_percent column is calculated before its usage
        df['short_exempt_percent'] = df['shortexemptvolume'] / df['totalvolume']

        # Calculate net short volume and net short shares
        df['net_short_volume'] = df['shortvolume'] - df['longvolume']
        df['net_short_shares'] = investment_amount / df['close']

        # Calculate cumulative net short volume and short exempt volume
        df['cumulative_net_short'] = (df['net_short_volume'] * df['net_short_shares']).rolling(window=days).sum()
        df['cumulative_short_exempt'] = (df['short_exempt_percent'] * df['net_short_shares']).rolling(window=days).sum()

        # Calculate the cumulative values as a percent of total volume
        df['cumulative_net_short_percent'] = df['cumulative_net_short'] / (df['totalvolume'] * df['net_short_shares']).rolling(window=days).sum() * 100
        df['cumulative_short_exempt_percent'] = df['cumulative_short_exempt'] / (df['totalvolume'] * df['net_short_shares']).rolling(window=days).sum() * 100

        if include_diff:
            periods = [1, 5, 10, 21, 42]
            for i in range(len(periods)):
                for j in range(i + 1, len(periods)):
                    df[f'cumulative_net_short_percent_{periods[i]}_div_{periods[j]}'] = df['cumulative_net_short_percent'].rolling(window=periods[i]).mean() / df['cumulative_net_short_percent'].rolling(window=periods[j]).mean()
                    df[f'cumulative_short_exempt_percent_{periods[i]}_div_{periods[j]}'] = df['cumulative_short_exempt_percent'].rolling(window=periods[i]).mean() / df['cumulative_short_exempt_percent'].rolling(window=periods[j]).mean()

        if include_roc:
            for col in ['cumulative_net_short_percent', 'cumulative_short_exempt_percent']:
                df[f'{col}_roc'] = df[col].pct_change() * 100

        selected_cols = ['cumulative_net_short_percent', 'cumulative_short_exempt_percent']
        if include_diff or include_roc:
            selected_cols += [col for col in df.columns if 'cumulative_net_short_percent' in col or 'cumulative_short_exempt_percent' in col]

        return df[selected_cols]

# Example usage 
analysis = ShortSaleVolume()
# df_gme_short_ratio = analysis.calculate_short_ratio_means('GME', include_roc = False)
# df_gme_short_sale_exempt = analysis.calculate_short_exempt_means('GME', include_roc = False)
# df_gme_investment_values = analysis.calculate_investment_values('GME', include_roc = False)


In [5]:
import psycopg2
import pandas as pd

class GreekAnalysis:
    def __init__(self):
        self.db_params = {
            'user': 'USER_STRING_HERE',    #removed for Git
            'password': 'PW_STRING_HERE',  #removed for Git
            'host': 'mads-capstone.cmohac77hep9.eu-north-1.rds.amazonaws.com',
            'port': 5432,
            'database': 'mads'
        }

    def get_greeks_data(self, ticker, greek_type):
        with psycopg2.connect(**self.db_params) as db_connection:
            db_connection.autocommit = True
            with db_connection.cursor() as cursor:
                select_query = "SELECT * FROM greeks3 WHERE ticker=%s AND GREEK=%s"
                cursor.execute(select_query, (ticker, greek_type))
                rows = cursor.fetchall()
                df = pd.DataFrame(rows, columns=["ID", "DATE", "INCREMENT", "VALUE", "TICKER", "GREEK", "OPTION_TYPE"])
        return df

    def analyze_data(self, df, greek_type):
        merged_df = self._prepare_merged_dataframe(df, greek_type)
        merged_df[f'{greek_type}_Skew'] = merged_df[f'{greek_type}_Call'] / merged_df[f'{greek_type}_Put']
        merged_df[f'{greek_type}_Time_Spread'] = merged_df[f'{greek_type}_30'] / merged_df[f'{greek_type}_90']
        return merged_df[[f'{greek_type}_30', f'{greek_type}_Skew', f'{greek_type}_Time_Spread']]

    def _prepare_merged_dataframe(self, df, greek_type):
        inc30_all = df[(df['INCREMENT'] == 30) & (df['OPTION_TYPE'] == 'all')]
        inc30_call = df[(df['INCREMENT'] == 30) & (df['OPTION_TYPE'] == 'call')]
        inc30_put = df[(df['INCREMENT'] == 30) & (df['OPTION_TYPE'] == 'put')]
        inc90_all = df[(df['INCREMENT'] == 90) & (df['OPTION_TYPE'] == 'all')]

        merged_df = pd.DataFrame(index=inc30_all['DATE'].unique())
        merged_df[f'{greek_type}_30'] = inc30_all.groupby('DATE')['VALUE'].mean()
        merged_df[f'{greek_type}_Call'] = inc30_call.groupby('DATE')['VALUE'].mean()
        merged_df[f'{greek_type}_Put'] = inc30_put.groupby('DATE')['VALUE'].mean()
        merged_df[f'{greek_type}_90'] = inc90_all.groupby('DATE')['VALUE'].mean()

        return merged_df

    def _add_rolling_features(self, df, greek_type):
        periods = [1, 5, 10, 21, 42]
        for col in [f'{greek_type}_30', f'{greek_type}_Skew', f'{greek_type}_Time_Spread']:
            for period in periods:
                df[f'{col}_rolling_{period}'] = df[col].rolling(window=period).mean()
        return df

    def _add_z_score_features(self, df, greek_type):
        rolling_window = 90
        for col in [f'{greek_type}_30', f'{greek_type}_Skew', f'{greek_type}_Time_Spread']:
            df[f'{col}_z_score'] = (df[col] - df[col].rolling(window=rolling_window).mean()) / df[col].rolling(window=rolling_window).std()
        return df

    def get_analysis_for_ticker(self, ticker, greek_type='iv', include_rolling=False, include_z_score=False):
        df = self.get_greeks_data(ticker, greek_type)
        analysis_df = self.analyze_data(df, greek_type)
        if include_rolling:
            analysis_df = self._add_rolling_features(analysis_df, greek_type)
        if include_z_score:
            analysis_df = self._add_z_score_features(analysis_df, greek_type)
        return analysis_df.sort_index()

# Example usage:
greek_analysis = GreekAnalysis()
# df_gme_iv = greek_analysis.get_analysis_for_ticker('GME', 'iv', include_rolling = True, include_z_score = True)
# df_gme_gamma = greek_analysis.get_analysis_for_ticker('GME', 'gamma', include_rolling = True, include_z_score = True)


In [6]:
class VolatilityAnalysis(GreekAnalysis):
    def __init__(self):
        super().__init__()
        self.sdate = '20160101'
        self.edate = '20231110'
        
    # Function to calculate HV for a given OHLCV DataFrame
    def calculate_hv(self, ohlcv_df, look_forward=30):
        # Calculate daily returns
        daily_returns = ohlcv_df['close'].pct_change()

        # Compute rolling standard deviation of daily returns
        rolling_std = daily_returns.rolling(window = look_forward).std()

        # HV is the annualized standard deviation
        hv = rolling_std * np.sqrt(252)

        return hv

    def compare_iv_hv(self, ticker, shift_hv = False):
        
        # Get ticker data
        ohlcv_df = finance.get_data(ticker, self.sdate, self.edate)
        
        # Pull IV30 data
        iv30_df = self.get_greeks_data(ticker, 'iv')
        iv30_df = iv30_df[(iv30_df['GREEK'] == 'iv') & (iv30_df['OPTION_TYPE'] == 'all') & (iv30_df['INCREMENT'] == 30)].copy()
        iv30_df['date'] = pd.to_datetime(iv30_df['DATE'])
        iv30_df.rename({'VALUE':'iv30'}, inplace = True, axis = 1)

        # Calculate HV
        hv_df = self.calculate_hv(ohlcv_df)

        # Align and compare IV30 and HV
        # Ensure iv30_df and hv_df have the same date index
        comparison_df = pd.DataFrame()
        comparison_df['IV30'] = iv30_df.set_index('date').sort_index()['iv30']
        
        # Are we shifting HV? 
        if shift_hv:
            comparison_df['HV'] = hv_df.shift(-30)  # Adjust as necessary
        else:
            comparison_df['HV'] = hv_df
        
        # Add to dataframe and return 
        comparison_df['IV_HV_Difference'] = comparison_df['IV30'] - comparison_df['HV']

        return comparison_df['IV_HV_Difference']
    
    def calculate_vwap(self, ohlcv_df, rolling_window = 90):  
        ohlcv_df['Typical_Price'] = (ohlcv_df['high'] + ohlcv_df['low'] + ohlcv_df['close']) / 3
        ohlcv_df['TP_Volume'] = ohlcv_df['Typical_Price'] * ohlcv_df['volume']

        # Calculate rolling TP_Volume and rolling volume
        ohlcv_df['Rolling_TP_Volume'] = ohlcv_df['TP_Volume'].rolling(window=rolling_window).sum()
        ohlcv_df['Rolling_Volume'] = ohlcv_df['volume'].rolling(window=rolling_window).sum()

        # Compute rolling VWAP
        ohlcv_df['VWAP'] = ohlcv_df['Rolling_TP_Volume'] / ohlcv_df['Rolling_Volume']

        # Calculate the percentage difference between close and VWAP
        ohlcv_df['vwap_close_pct'] = ohlcv_df['close'] / ohlcv_df['VWAP']

        return ohlcv_df

    def compare_price_to_vwap(self, ticker = None, ohlcv_df = None):
        
        # Optionally, grab data
        if not isinstance(ohlcv_df, pd.DataFrame):
            ohlcv_df = finance.get_data(ticker, self.sdate, self.edate)
            
        ohlcv_df = self.calculate_vwap(ohlcv_df)
        return ohlcv_df['vwap_close_pct']
    
# Example usage
price_iv_analysis = VolatilityAnalysis()
# df_iv_hv = price_iv_analysis.compare_iv_hv('GME')
# df_vwap = price_iv_analysis.compare_price_to_vwap('GME')

In [7]:
import requests
from bs4 import BeautifulSoup

def return_etf_holdings(ticker = 'SPY'):

    headers = requests.utils.default_headers()
    headers['User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
    res = requests.get("https://www.zacks.com/funds/etf/%s/holding"%ticker,  headers=headers)
    soup = BeautifulSoup(res.text, 'html.parser')

    js = None
    for script in soup.find_all("script"):
        if "etf_holdings.formatted_data = " in str(script):
    #    if "etf_holdings" in str(script):
            js = str(script)
    holdings = []
    # for line in js.text.split("["):
    for line in js.split("["):
        if "rel=" in line:
            line_soup = BeautifulSoup(line, 'html.parser')
            #holding = line_soup.find("a")["alt"].strip('\\"')
            holding = line_soup.find('a')['rel'][0].strip('\\"')
            holdings.append(holding)

    return holdings

In [8]:
tickers = return_etf_holdings('IWV')

In [9]:
len(tickers)

2662

In [10]:
# !pip install pandas_market_calendars

In [11]:
import pandas_market_calendars as mcal

# Define the date range
start_date = '2018-08-01'
end_date = '2023-11-15'

# Create a calendar for NYSE
nyse_calendar = mcal.get_calendar('NYSE')

# Get the open market days within the date range
market_days = nyse_calendar.valid_days(start_date=start_date, end_date=end_date)

# Convert to a list of dates in string format (if needed)
market_days_str = market_days.strftime('%Y-%m-%d').tolist()


# Create enormous dataframe

In [ ]:
from tqdm import tqdm
sdate = '20180816'
edate = '20231115'

df_repo = []
for ticker in tqdm(tickers):
    
    try:
        # Get data
        df_data = get_data(ticker, sdate, edate)

        # Calc positive and negative classes
        price_jumps_df = identify_price_jumps(df_data, z_score_threshold=2.5, stability_threshold=1, include_negative_class=True, remove_duplicate_price_jump_dates = False)

        # Get other data
        df_short_ratio = analysis.calculate_short_ratio_means(ticker, include_diff=True, include_roc=True)
        df_short_sale_exempt = analysis.calculate_short_exempt_means(ticker, include_diff=True, include_roc=True)
        df_investment_values = analysis.calculate_investment_values(ticker, include_diff=True, include_roc=True)

        df_iv = greek_analysis.get_analysis_for_ticker(ticker, 'iv', include_rolling = True, include_z_score = True)
        df_gamma = greek_analysis.get_analysis_for_ticker(ticker, 'gamma', include_rolling = True, include_z_score = True)

        df_iv_hv = price_iv_analysis.compare_iv_hv(ticker)
        df_vwap = price_iv_analysis.compare_price_to_vwap(ticker)

        # Merge datasets together
        additional_dfs = [df_short_ratio, df_short_sale_exempt, df_investment_values, df_iv, df_gamma, df_iv_hv, df_vwap]
        for df_ in additional_dfs:
            df_.index = pd.to_datetime(df_.index)
        merged_df = merge_additional_data(price_jumps_df, additional_dfs)

        # Save to repo and continue
        df_repo.append(merged_df)
        
    except Exception as e:
        print(ticker, type(e), e)

  0%|          | 0/2662 [00:00<?, ?it/s]

AAPL <class 'requests.exceptions.ConnectionError'> ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


/tmp/ipykernel_9821/323460992.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_z_score = (df.iloc[i]['PriceChange'] - rolling_mean[i]) / rolling_std[i]
/tmp/ipykernel_9821/323460992.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  future_z_score = (future_price_change - rolling_mean[i+lag_day]) / rolling_std[i+lag_day]
/opt/conda/lib/python3.10/site-packages/fsspec/registry.py:272: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update t